# Purpose of This Project
This project will contain python scripts for the purposes of simulating and analyzing GURPS 4e combat mechanics. I plan to take a functional approach to this calculator to practice my functional programming in python. I may or may not break down and cry and write for loops instead. I expect to embarrassingly mess up the whole adhering to the functional paradigm entirely and have to revisit a lot of my functions. Either way, I'll write it all down in a Jupyter notebook to document the journey.

First, I'll start by finding a function that can roll dice. GURPS 4e combat is highly reliant on dice rolls, so—while we can analyze the rolls mathematically—the ability to run simulations should be a lot more practical in situations where we are learning the mathematics or want to see how it might feel to a player in our game.

We'll use the random module

In [1]:
import random as rd

print(rd.randint(3,18))

12


GURPS 4e heavily utilizes the roll of three six sided dice (3d6), abbreviated in the context of the system simply to 3d. Our 3d rolls will have a minimum of 3 and a maximum of 18, as shown above, but the distribution won't be completely linear. We might repackage the `rd.randint()` function into another function. On that note, regarding the functional approach I'd like to take, I am not going to remake `rd.randint()` from scratch to make it more "functional". This approach practice will only apply to user-defined functions for the time being.

First, we should aim to make a couple of functions: one to roll any number of dice and output a number, particularly rolls of 3d, and one that calculates the probability of getting any given number from a roll.

We are going to assume that every single dice we work with will be one that starts at 1, and counts up one whole number until there is a unique number on every face of an n-sided die from 1 to n.

# Dice Roll Functions

In [2]:
def roll_dx(x):
    return rd.randint(1,x)

def roll_xd6(x):
    return x*roll_dx(6)

print(f'roll 3d6 5 times: {roll_xd6(3)}, {roll_xd6(3)}, {roll_xd6(3)}, {roll_xd6(3)}, {roll_xd6(3)}')
print(f'roll 5d6 5 times: {roll_xd6(5)}, {roll_xd6(5)}, {roll_xd6(5)}, {roll_xd6(5)}, {roll_xd6(5)}')

roll 3d6 5 times: 3, 12, 12, 12, 18
roll 5d6 5 times: 10, 30, 5, 25, 25


The way that our `roll_dx()` function is written, it will give an error for any input besides whole numbers greater than or equal to `1`.

# Dice Probability Functions

Our first dice probability function will be a lot like our function `roll_dx` above. The input for the function will be a whole number greater than or equal to `1` that says how many sides we have on our dice. The output will be a dictionary of all the possible outcomes as keys and their probabilities as values.

## Coming up with a solution
the first approach that comes to mind is to take our aforementioned input as a local variable `x`, then create an empty dictionary to add entries to. In the case of an input that is `6`, we would want our function to add numbers from 1 to 6 to our dictionary, and make the value a probability in the form of a calculation: 1/6 for each one.

I'm doing some lame empty dict technique for this, so it may seem like I have given up on making this hardcore functional and you are correct. The reason is that I have a baby to take care of and she doesn't have the patience for dad to learn how to do this. Refactoring our functions will come at a later date.

In [3]:
def prob_dx(x):
    d = {}
    for i in range(x):
        d[i+1] = 1/x
    return d

print(prob_dx(6))

{1: 0.16666666666666666, 2: 0.16666666666666666, 3: 0.16666666666666666, 4: 0.16666666666666666, 5: 0.16666666666666666, 6: 0.16666666666666666}


Supposedly it is ok to use a dictionary and return one, as long as we don't store it anywhere, creating some mutable state. We could force the output of the function to be an iterator, but I think we are adults and can decide whether or not we want to use our functions as functionally as possible.

# WHAT IS FUNCTIONAL PROGRAMMING

For the amount I've been mentioning functional programming, I sure haven't defined what it is.

For the purposes of this project, we will be using [this](https://docs.python.org/3/howto/functional.html) page as a standard for what functional programming in python is. This howto emphasizes modularity, composability, and ease of debugging and testing.

# Dice Probability Functions cont.

Let's try to refactor our function from above with a dictionary comprehension instead of all the lines we used

In [4]:
def prob_dx(x):
    return {n+1:1/x for n in range(x)}

print(prob_dx(6))

{1: 0.16666666666666666, 2: 0.16666666666666666, 3: 0.16666666666666666, 4: 0.16666666666666666, 5: 0.16666666666666666, 6: 0.16666666666666666}


nice and sleek

Next, we'll want something that can give us the probability distribution of multiple dice added together in the same roll, such as 3d6. The resulting dictionary in the case of 3d6 should have values 3 through 18 and their probabilities as a number from 0 to 1. If we wanted to calculate this, we might list the number of permutations possible with 3d6 and figure out how many combinations out of 216 give a certain value.

I have a feeling that, instead of doing that, we can write a function that does this on the fly while doing math with dictionaries, so let's try.

## Finding a solution for 3d6 probability calculator

We would want to add up our keys to get new keys: 1+1+1=3, 1+1+2=4, etc. sounds like for loop hell that I may or may not refactor into a comprehension.

How do the values factor in? well, in the case of 1+1+1=3, we want to end up doing some operation with 1/6, 1/6, and 1/6 that comes out to 1/216 or 1/(6^3). It seems obvious that we will want to multiply our probabilities together. Let's check 1+1+2: that gives the same exact result, but our for loop hell will run into 4 examples of this. How do we add the probabilities together when we calculate a key that is already in use?

We can reference the previous value for the key and add our result to it to set the value of the key. This is poopy mutability, but I'll survive for now. I just have to figure out how to make this work for setting the key in the first place.

`dict[1+1+1] += 1/6 * 1/6 * 1/6` works if `dict[1+1+1]` already equals 0. But IDK if that works of if I have to set the key value to 0 first.

...

I just checked and yeah you can't set a dictionary entry with `+=`

so one solution is to set up all of the keys and have them equal 0 ahead of time, and the other is to do a for loop hell before this one to set up the keys for the dictionary. Both of these suck.

After much searching I found the .get() method for dictionaries. It is identical to using a key as an index except you can't assign anything to it, and, importantly, you can set a default value for keys that don't exist yet. So that gives me a great way to do something like += except being able to start off without a key via default=0.

In [9]:
def prob_xd6(x):
    d = {}
    for n, m in prob_dx(6).items():
        for j, k in prob_dx(6).items():
            for i, o in prob_dx(6).items():
                d[n+j+i] = d.get(n+j+i, 0) + (m*k*o)
    return d
                
print(prob_xd6(3))

{3: 0.004629629629629629, 4: 0.013888888888888888, 5: 0.027777777777777776, 6: 0.046296296296296294, 7: 0.06944444444444445, 8: 0.09722222222222218, 9: 0.11574074074074067, 10: 0.12499999999999992, 11: 0.12499999999999992, 12: 0.11574074074074067, 13: 0.09722222222222218, 14: 0.06944444444444445, 15: 0.046296296296296294, 16: 0.027777777777777776, 17: 0.013888888888888888, 18: 0.004629629629629629}
